```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2021
Profesor: Facundo Santiago, Javier Ignacio Garcia Fronti
```

# ¿Qué tan memorable es un video?: Solución base utilizando videos, images y audio

Este notebook les permitirá tener a disposición todos los recursos que se vieron en durante la materia. Todos los fragmentos de código se descargaran utilizando la sección preparación del ambiente.

## Preparación del ambiente

### Sets de datos

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/audio_vectors.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/c3d_vectors.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/caption_features.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/colors_features.csv --directory-prefix ./Data/Features/

### Librerías de codigo que se utilizan durante el curso

#### Audio

In [ ]:
!wget -N https://storage.googleapis.com/audioset/yamnet.h5 --directory-prefix ./Models/yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Models/yamnet/yamnet_class_map.csv --directory-prefix ./Models/yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Utils/audio_plotter.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/yamnet.py --directory-prefix ./yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/params.py --directory-prefix ./yamnet/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/features_lib.py --directory-prefix ./yamnet/

In [ ]:
!pip install librosa
!pip install moviepy
!pip install soundfile
!pip install azure-cognitiveservices-speech

#### NLP

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextNormalizer.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py --directory-prefix ./Utils/
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin --directory-prefix ./Models/Word2Vec

In [ ]:
!pip install transformers
!pip install unidecode
!python -m spacy download es_core_news_sm

#### Vision

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Vision/Utils/Cognitive.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Vision/Utils/Plotting.py --directory-prefix ./Utils/

In [ ]:
!pip install tensorflow-datasets
!pip install azure-cognitiveservices-vision-computervision
!pip install matplotlib
!pip install skimage

### Imports

In [ ]:
import pandas as pd
import numpy as np
import librosa
import os
import soundfile as sf
import torch
import transformers
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D
import tensorflow_datasets as tfds

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from yamnet.yamnet import yamnet_frames_model
import yamnet.yamnet as yamnet_model
from yamnet.params import Params

from Utils.audio_plotter import plot_audio_embeddings
from Utils.TextDataset import ClassificationDataset
from Utils.TextNormalizer import TweetTextNormalizer
from Utils.Word2VecVectorizer import Word2VecVectorizer
from Utils.PadSequenceTransformer import PadSequenceTransformer

import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

## Solución

Cuentan con 2 tipos de anotaciones para cada uno de los fragmentos de video disponibles:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular, desde 0 a 1. Valores más grandes son mejores.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con `memorability_score` superior a `0.5` es marcado como memorable (`1`), sino es marcado como no memorable (`0`)

In [ ]:
labels = pd.read_csv('Data/ground_truth.csv')

In [ ]:
labels.head(2)

,movie_name,start(sec),end(sec),sequence_name,Neutral (1)_Typical (0),nb_annotations,memorability_score,memorabable
0,127 hours,2000,2010,127_hours_2000_2010_1,0,5,1.0,1
1,127 hours,2182,2192,127_hours_2182_2192_5,1,8,0.0,0


> Note que aquí nuestras muestras son "secuencias" de determinadas películas. En total dispone de 609 secuencias con el nombre `sequence_name`. El mismo nombre se generó automáticamente concatenando el nombre de la pelicula a la que pertenece la secuencia (movie_name), seguido del segundo en el que comienza la secuencia, seguido del segundo en el que termina, seguido de un numero que indica el número de secuencia. Por ejemplo, la secuencia `127_hours_2000_2010_1` es un fragmento de la pelicula "127 hours", que va desde el segundo 2000 (00:33:20 hrs.) al segundo 2010 (00:33:30 hrs.) y es el fragmento número 1. Esta información es totalmente irrelevante para el problema de memorabilidad.

### Datos no estructurados

Dado que los conjuntos de datos pueden ser de gran tamaño, descargue solo los datos que necesita:

#### Para utilizar los cuadros (frames)

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/frames.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/frames.tar.gz --directory ./Data/Raw/

> En el directorio `Data/Raw/frames` encontraran todos los cuadros extraidos de cada una de las secuencias. Los cuadros están disponibles cada 2 segundos por lo cual disponen de 5 cuadros por cada secuencia. Todos los cuadros pertenecientes a una secuencia están almacenados en una carpeta con el mismo nombre de la secuencia a la que pertenece, por lo que la carpeta `127_hours_2000_2010_1` contiene todos los cuadros pertenecientes a la secuencia `127_hours_2000_2010_1` en el conjunto de datos de anotaciones.

In [ ]:
# completar solución

#### Para utilizar los audios

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/audios.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/audios.tar.gz --directory ./Data/Raw/

> En el directorio `Data/Raw/audios` encontraran todos los audios correspondientes a cada una de las secuencias de los videos. Cada audio tiene el mismo nombre de la secuencia a la que pertenece, por lo que el archivo `127_hours_2000_2010_1.wav` pertenece a la secuencia `127_hours_2000_2010_1` en el conjunto de datos de anotaciones. Los audios están en formato `wav`, que si bien ocupan mayor espacio, son más sencillos de procesar.

In [ ]:
# completar solución

#### Para utilizar los videos

In [ ]:
!wget -N https://santiagxf.blob.core.windows.net/public/Memorability/sources.tar.gz --directory-prefix ./Data/Raw/
!tar zxvf ./Data/Raw/sources.tar.gz --directory ./Data/Raw/

> En el directorio `Data/Raw/sources` encontraran todas las secuencias utilizadas en este desafío. El archivo `Data/ground_truth.csv` contine los nombres de las peliculas a las que corresponde cada fragmento en caso que quieran revisarlos. Cada video tiene el mismo nombre de la secuencia a la que pertenece, por lo que el archivo `127_hours_2000_2010_1.mp4` pertenece a la secuencia `127_hours_2000_2010_1` en el conjunto de datos de anotaciones.

In [ ]:
# completar solución